In [8]:
import numpy as np
import open3d as o3d
import plotly.graph_objs as go

x = np.loadtxt("Datos/array_x1.txt")
y = np.loadtxt("Datos/array_y1.txt")
z= np.loadtxt("Datos/array_z1.txt")


In [9]:
#Combina los arrays en una matriz
xyz = np.column_stack((x,y,z))

# Crea un objeto PointCloud de Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

# Guarda una nube de puntos en formato PLY
o3d.io.write_point_cloud("nube_puntos.ply", pcd)

print("Nube de puntos guardada")

Nube de puntos guardada


In [10]:

pcd = o3d.io.read_point_cloud("nube_puntos.ply")
points = np.asarray(pcd.points)

# Visualiza con Plotly
scatter = go.Scatter3d(
    x=points[:, 0], y=points[:, 1], z=points[:, 2],
    mode='markers',
    marker=dict(size=2, color=x, colorscale='Viridis', opacity=0.8)
)

layout = go.Layout(scene=dict(aspectmode='data'))
fig = go.Figure(data=[scatter], layout=layout)
fig.show()

In [5]:
pcd = o3d.io.read_point_cloud("nube_puntos.ply")
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [11]:
# Clustering con KMeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

n_clusters = 6
kmeans = KMeans(n_clusters=n_clusters, random_state=93,n_init='auto')

labels = kmeans.fit_predict(points)


# Visualiza los clusters con Plotly
scatter = go.Scatter3d(
    x=points[:, 0], y=points[:, 1], z=points[:, 2],
    mode='markers',
    marker=dict(size=2, color=labels, colorscale='Turbo', opacity=0.8)
)
layout = go.Layout(
    title = f'KMeans CLustering (k={n_clusters})',
    scene=dict(aspectmode='data')
    )
fig = go.Figure(data=[scatter], layout=layout)
fig.show()

In [21]:
from sklearn.cluster import DBSCAN

#Parametros para DBSCAN
eps = 3
min_samples = 5

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(points)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Numero de clusters encontrados: {n_clusters}")

scatter = go.Scatter3d(
    x=points[:, 0], y=points[:, 1], z=points[:, 2],
    mode='markers',
    marker=dict(size=2, color=labels, colorscale='Turbo', opacity=0.8)
)
layout = go.Layout(
    title=f'DBSCAN Clustering (eps={eps}, min_samples={min_samples})',
    scene=dict(aspectmode='data')
)
fig = go.Figure(data=[scatter], layout=layout)
fig.show()

Numero de clusters encontrados: 17


In [22]:
from sklearn.cluster import OPTICS

# Parámetros de OPTICS
min_samples = 10     
xi = 0.05            # Umbral de clusterización
min_cluster_size = 0.05  # Proporción del total o número absoluto

# Aplica OPTICS
optics = OPTICS(min_samples=min_samples, xi=xi, min_cluster_size=min_cluster_size)
labels = optics.fit_predict(points)

# Número de clusters detectados (excluyendo ruido)
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f'Número de clusters detectados por OPTICS: {n_clusters}')

# Visualización
scatter = go.Scatter3d(
    x=points[:, 0], y=points[:, 1], z=points[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=labels,
        colorscale='Plasma',
        opacity=0.8
    )
)

layout = go.Layout(
    title=f'OPTICS Clustering (min_samples={min_samples}, xi={xi})',
    scene=dict(aspectmode='data')
)
fig = go.Figure(data=[scatter], layout=layout)
fig.show()


Número de clusters detectados por OPTICS: 3
